# Project Title
Sentiment analysis on twitter with nltk. 

## Members

Betül Durkaya, durkaya@itu.edu.tr

## Description of the project
Sentiment analysis the measurement of positivity and negativity of opinions about a topic. Also, NLTK(natural language toolkit) is a tool including libraries and programs for natural language processing in python.

### The methods to be used

Python is going to be used for the project. For access to twitter, we must obtain some keys by help twitter API. We are going to request with codes to connect twitter; furthermore, the connection is going to be actualized owing to tweepy library and aceess keys. In addition, we are going to used json library; accordingly, the response from twitter is in json format. In the beginning, the json data is going to be collected in a txt file. Then, we are going to save the data in a database(MongoDB). After that, we are going to analyzed the data by using a python library that is called NLTK.


### The data

The data is tweets from twitter. Tweets are going to gather in json format.

## Code


In [1]:
import tweepy
from tweepy import OAuthHandler
import listening

#twitter api keys to authanticate
consumer_key = "dtKTW35ISxLtPDCszilhNk6R2"
consumer_secret = "pDoisuGvzTGYte5rNEtjguiixVyys0IrL98xu5tzxE7aG8KZCJ"

access_token = "773843946-LBFza4JmUOrzrUOkahDdtHXi6AHwFQyjwH6j7Ibg"
access_token_secret = "JU0ExxNVFkQbBzipW0EOL9Lkb6dm3RadGdfoJeaLdOUgm"

#authentication sections
auth = OAuthHandler(consumer_key, consumer_secret)
print("First auth done")
auth.set_access_token(access_token, access_token_secret)
print("2nd auth done")

#tracked word
WORDS = ['iphone']

#from listening library, StreamListener object created
listener = listening.StreamListener(api=tweepy.API(wait_on_rate_limit=True))
#from tweepy library, Stream begins
streamer = tweepy.Stream(auth=auth, listener=listener)

print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS, languages=["en"]) #filtering keywords

First auth done
2nd auth done
Tracking: ['iphone']
Tweet streaming begin.

 ('Kiogco_Liunho', 'Creative 3D Game Boy Silicon Skin Back Case Cover Protector For iPhone 5 / 5S https://t.co/Ghkr5FRGbW https://t.co/wofTIBFKRX') 





 ('Xeejdo_Xoubvu', 'Creative 3D Game Boy Silicon Skin Back Case Cover Protector For iPhone 5 / 5S https://t.co/USFSOTANuZ https://t.co/jD6ta2Y5iF') 





 ('Kiogco_Liunho', 'LIFEPROOF FRĒ BLACK CASE IPHONE 5c New In Box Sealed  https://t.co/pyfQo3u9bm https://t.co/LeyB0dFpPH') 





 ('Xeejdo_Xoubvu', 'LIFEPROOF FRĒ BLACK CASE IPHONE 5c New In Box Sealed  https://t.co/qdgLlG4Lx3 https://t.co/XSdzV5LGH0') 





 ('Kiogco_Liunho', 'NO RESERVE! iPhone 5 + charger + tempered glass + case black 16G bundle https://t.co/1S5UyhrGJB https://t.co/nyet41G3Be') 





 ('Maddcloner', "@joepabike @realDonaldTrump @foxandfriends @POTUS let's hope he can fix the mess Obama left behind without tapping my IPhone like Obama did") 





 ('Haayme_Faayho', 'Creative 3D Game Boy Sil

KeyboardInterrupt: 

In [ ]:

from pymongo import MongoClient
import tweepy
import json

#MongoDB connection is established and configured
connection = MongoClient('localhost', 27017)
db = connection.StreamData 
db.tweets.ensure_index("id", unique=True, dropDups=True)
collection = db.tweets

#organizing tweet information
class StreamListener(tweepy.StreamListener):

    def on_connect(self):
        print("Tweet streaming begin.")

    def on_error(self, status):
        print('Error Type: ' + status)
        return False

    def on_data(self, raw):
        data = json.loads(raw)
        tweet_id = data['id_str']  # The ID of tweet from Twitter in string format
        username = data['user']['screen_name']  # The Tweet author's username
        text = data['text']  # The entire body of the Tweet

        try:
            print('\n', (data["user"]["screen_name"], data["text"]), '\n\n\n\n')
            if data["text"].find('RT @') is -1: #if not exist
                tweet = {'id': tweet_id, 'username': username, 'text': text}
                # Insert Tweet data to MongoDB
                collection.save(tweet)
        except Exception as e:
                    print(e)
        return True